In [1]:
!pip install python-telegram-bot==13.15 PyDrive2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.4/497.4 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 2.1 MB/s eta 0:00:00
  Created wheel for tornado: filename=tornado-6.1-cp311-cp311-linux_x86_64.whl size=422005 sha256=03249c233b5064984eb25bce6bfe87d7aba536562e289c4e7c178f66828d9b1b
  Stored in directory: /root/.cache/pip/wheels/f2/59/06/a9c85c7b17ec0fc9b1e2ae0c59e3d39255d5c0a38492e33fea
Successfully built tornado
  Attempting uninstall: tornado
    Found existing installation: tornado 6.4.2
    Uninstalling tornado-6.4.2:
      Successfully uninstalled tornado-6.4.2
  Attempting uninstall: cachetools
    Found existing installation: cachetools 5.5.2
    Uninstalling cachetools-5.5.2:
      Successfully uninstalled cachetools-5.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are in

In [2]:
import logging, tempfile
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth

# Google authentication for Colab
auth.authenticate_user()

# 🎯 Replace with your actual credentials
CLIENT_ID = "YOUR_GOOGLE_CLIENT_ID"
CLIENT_SECRET = "YOUR_GOOGLE_CLIENT_SECRET"

# PyDrive2 authentication using provided credentials
gauth = GoogleAuth()
gauth.DEFAULT_SETTINGS['client_config_backend'] = 'settings'
gauth.DEFAULT_SETTINGS['client_config'] = {
    "installed": {
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "auth_uri": "https://accounts.google.com/o/oauth2/auth",
        "token_uri": "https://oauth2.googleapis.com/token"
    }
}
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)

# Logging
logging.basicConfig(format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO)

# Telegram bot handlers
def start(update: Update, context: CallbackContext):
    update.message.reply_text("📂 Send me a file, I will upload to Google Drive!")

def handle_docs(update: Update, context: CallbackContext):
    doc = update.message.document.get_file()
    name = update.message.document.file_name
    with tempfile.NamedTemporaryFile(delete=False) as tf:
        doc.download(custom_path=tf.name)
        gfile = drive.CreateFile({"title": name})
        gfile.SetContentFile(tf.name)
        gfile.Upload()
        gfile.InsertPermission({"type": "anyone", "value": "anyone", "role": "reader"})
        update.message.reply_text(f"✅ Uploaded! {gfile['alternateLink']}")

def main():
    TOKEN = "YOUR_TELEGRAM_BOT_TOKEN"
    updater = Updater(token=TOKEN, use_context=True)
    dp = updater.dispatcher
    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(MessageHandler(Filters.document, handle_docs))
    updater.start_polling()
    updater.idle()

if __name__ == "__main__":
    main()

MessageError: Error: credential propagation was unsuccessful

In [3]:
!pip install python-telegram-bot==13.15
!pip install PyDrive

In [4]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
print("✅ Google Drive authenticated successfully.")

✅ Google Drive authenticated successfully.


In [5]:
import logging
from telegram import Bot, Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
import tempfile
from pydrive.drive import GoogleDrive

# Logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)

# آپ کا Bot Token
bot_token = "YOUR_TELEGRAM_BOT_TOKEN"

# Updater & Dispatcher Setup
updater = Updater(token=bot_token, use_context=True)
dispatcher = updater.dispatcher

# Start Command Function
def start(update: Update, context: CallbackContext):
    update.message.reply_text("📤 Send me any document and I’ll upload it to Google Drive!")

# Handle Document Upload
def handle_docs(update: Update, context: CallbackContext):
    file = update.message.document.get_file()
    file_name = update.message.document.file_name

    with tempfile.NamedTemporaryFile(delete=False) as tf:
        file.download(custom_path=tf.name)
        gfile = drive.CreateFile({'title': file_name})
        gfile.SetContentFile(tf.name)
        gfile.Upload()
        gfile.InsertPermission({'type': 'anyone', 'value': 'anyone', 'role': 'reader'})
        file_link = gfile['alternateLink']

    update.message.reply_text(f"✅ Uploaded Successfully!\n🔗 [View File]({file_link})", parse_mode='Markdown')

# Register Handlers
dispatcher.add_handler(CommandHandler("start", start))
dispatcher.add_handler(MessageHandler(Filters.document, handle_docs))

# Start the bot
print("🤖 Bot is running... Waiting for documents.")
updater.start_polling()

🤖 Bot is running... Waiting for documents.
